<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#A-TOUR-OF-HADOOP-ECOSYSTEM-THROUGH-HORTONWORKS-SANDBOX" data-toc-modified-id="A-TOUR-OF-HADOOP-ECOSYSTEM-THROUGH-HORTONWORKS-SANDBOX-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>A TOUR OF HADOOP ECOSYSTEM THROUGH HORTONWORKS SANDBOX</a></span><ul class="toc-item"><li><span><a href="#Simple-Data-import-and-Map-Reduce-Job" data-toc-modified-id="Simple-Data-import-and-Map-Reduce-Job-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Simple Data import and Map Reduce Job</a></span><ul class="toc-item"><li><span><a href="#Data-Import" data-toc-modified-id="Data-Import-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Data Import</a></span></li><li><span><a href="#Map-Reduce-Job" data-toc-modified-id="Map-Reduce-Job-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>Map Reduce Job</a></span><ul class="toc-item"><li><span><a href="#Word-Count-on-NCDC-Dataset" data-toc-modified-id="Word-Count-on-NCDC-Dataset-1.1.2.1"><span class="toc-item-num">1.1.2.1&nbsp;&nbsp;</span>Word Count on NCDC Dataset</a></span></li><li><span><a href="#Title-Count-Across-Years-Using-IMDB-Dataset" data-toc-modified-id="Title-Count-Across-Years-Using-IMDB-Dataset-1.1.2.2"><span class="toc-item-num">1.1.2.2&nbsp;&nbsp;</span>Title Count Across Years Using IMDB Dataset</a></span></li></ul></li></ul></li><li><span><a href="#Creating-Scripts-for-MapReduce-Job" data-toc-modified-id="Creating-Scripts-for-MapReduce-Job-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Creating Scripts for MapReduce Job</a></span><ul class="toc-item"><li><span><a href="#Maximum-Temperature-Example-on-NCDC-Dataset" data-toc-modified-id="Maximum-Temperature-Example-on-NCDC-Dataset-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Maximum Temperature Example on NCDC Dataset</a></span></li></ul></li></ul></li></ul></div>

# A TOUR OF HADOOP ECOSYSTEM THROUGH HORTONWORKS SANDBOX

Hortonworks' "Sandbox" and Cloudera's "Clusterdock" are two self-contained docker images, that can run a full hadoop distribution in the stand-alone or semidistributed mode

Both of them are installed in Lab 409 PC's

However for ease of use, its open-source nature, the variety of Hadoop services available out of the box and focus on teaching Hadoop, we will prefer Hortonworks Sandbox

Hortonworks Sandbox features a standalone version of the full "Hortonworks Data Platform" (HDP) Hadoop distribution

In [ ]:
# start_sandbox-hdp.sh

It will take several minutes to make the sandbox up and running

You can communicate with the Sandbox through localhost:8888

In [1]:
firefox localhost:8888 &

[1] 12542


When you hit the "Launch Dashboard" button, the login screen for Ambari will appear.

Ambari is a completely open source management platform for provisioning, managing, monitoring and securing Apache Hadoop clusters.

The credentials are: admin, bda505

From the ambari dashboard, you can access:
- HDFS
- YARN
- MapReduce2
- Hive
- Hbase
- Pig
- Sqoop
- Spark2
- Zeppelin

and other projects within the Hadoop ecosystem.
We will see the use cases of all

The pop-up window features a tutorial series for using HDP

You can follow the link to watch a video about Introduction to Sandbox:

[Hadoop Tutorial: Introduction to Hortonworks Sandbox](https://www.youtube.com/watch?v=H0KXnfE9Z9s&list=PL2y_WpKCCNQcABNHVSwUwwK169xxwSdNp)

We can connect to our docker container using ssh.

For this please open a shell screen and enter the following command (without the leading #). The ssh password of root is set as "hadoopbda505"

Hopefully, passwordless ssh login is enabled as of this class:

In [1]:
# ssh root@localhost -p 2223

Now we show a simple data import and map reduce job:

## Simple Data import and Map Reduce Job

### Data Import

In order to import data into sandbox, we have to send the data to the docker container through ssh

Data are sent to the docker cotainer from the main system through scp or rsync commands

4 datasets are pushed to the docker container:

- The imdb dataset as tsv files
- A portion of the 2015 UN COMTRADE dataset (international trade statistics) as json files 
- 1901-1950 portion of the ncdc dataset (US weather statistics) as plain text files
- A portion of the google ngrams dataset (frequency of words in publications indexed by Google Books, by year of publication) as plain text files

The bash commands to create the necessary directories inside the docker container are as follows

Note that, this command is executed from the main system, not from inside the docker. It is executed via ssh:

In [1]:
# create data directories
#ssh root@localhost -p 2223 'yum -y install expect;
#if [ ! -d /data ]; then mkdir /data; fi;
#if [ ! -d /data/imdb ]; then mkdir /data/imdb; fi;
#if [ ! -d /data/comtrade ]; then mkdir /data/comtrade; fi;
#if [ ! -d /data/comtrade/gz ]; then mkdir /data/comtrade/gz; fi;
#if [ ! -d /data/ncdc ]; then mkdir /data/ncdc; fi;
#if [ ! -d /data/ncdc/gz ]; then mkdir /data/ncdc/gz; fi;
#if [ ! -d /data/ngrams ]; then mkdir /data/ngrams; fi;
#if [ ! -d /data/ngrams/gz ]; then mkdir /data/ngrams/gz; fi;'


And here we push the data into the container:

In [2]:
#rsync -e 'ssh -p 2223' -a /home/bda505/mef/01/Session_01_dataset/tsv/*2.tsv root@localhost:/data/imdb/;
#rsync -e 'ssh -p 2223' -a /home/bda505/mef/04/comtrade_2015/gz/2015_{1..152}_* root@localhost:/data/comtrade/gz;
#rsync -e 'ssh -p 2223' -a /home/bda505/mef/06/ncdc/{1901..1950}* root@localhost:/data/ncdc/gz;
#rsync -e 'ssh -p 2223' -a /home/bda505/mef/07/ngrams/googlebooks-eng-all-1gram-20120701-*[0-9a].gz root@localhost:/data/ngrams/gz;

#ssh root@localhost -p 2223 'if [ ! -d /data/comtrade/json ]; then cp -r /data/comtrade/gz /data/comtrade/json;\
#cd /data/comtrade/json; gunzip *.gz; fi;

#if [ ! -d /data/ncdc/txt ]; then cp -r /data/ncdc/gz /data/ncdc/txt;\
#cd /data/ncdc/txt; gunzip *.gz; fi;

#if [ ! -d /data/ngrams/txt ]; then cp -r /data/ngrams/gz /data/ngrams/txt;\
#cd /data/ngrams/txt; gunzip *.gz; fi

#chmod -R 777 /data

And the codes ...

In [10]:
# rsync -e 'ssh -p 2223' -a /home/bda505/mef-bigdata/week_07/codes root@localhost:/data/

Over network connections, and a large count of files that my change in time, rsync is a better choice. It is small, fast yet versatile: Supports many different options.

Rsync alone can be a perfect tool for backing up and synchronization even over remote connections

For a smaller number of files, scp may also be used:

In [ ]:
# scp -P 2223 /path/to/localfile root@localhost:~/

Now a little task for you:
- First open a shell window
- And open a second shell window
- With "ssh root@localhost -p 2223" command, login to the docker container, so that you can view what is inside
- In your "real" shell window - not the docker shell - download the 2014 flight data for NYC airports via:
"wget https://raw.githubusercontent.com/wiki/arunsrinivasan/flights/NYCflights14/flights14.csv"
- "wget" is also a very light, powerful and versatile command: You can even use it as a spider to download and create a mirror of a complete website!
- Now push the csv file into the docker container and confirm it is inside the docker container (using scp or rsync)
- You may create a separate directory inside the docker container
- You may use "ls" command to list directory contents

You can also access the shell inside the docker using the  shell web client or shell-in-a-box at localhost:4200

Use the credentials: root, hadoopbda505

We may use shell commands for some tasks, and ambari web ui for other purposes

The login details for ambari ui at localhost:8888 is admin, bda505

If you cannot login with this password, use the below command from inside the docker, to input your new password:

In [3]:
# ambari-admin-password-reset

We can upload data into HDFS either using ambari web ui or command line

Now, let's first see what our HDFS directories look like

From localhost:8888, click on dashboard, and login with admin, bda505

Just to the left of the top right corner, select "Files View"

That shows the contents of the root directory of HDFS

You can create folders or upload files from your "main" system - that's the easy way

But for the time being, we will upload data using command line

Inside docker shell or shell web client, change the user to hdfs, so that we can access the "hdfs":

In [ ]:
# su hdfs

And cd to root

In [5]:
# cd /

You can see the data directory that we pushed to the docker container with ls

hdfs commands are prefixed with "hdfs dfs" followed by a dash and ordinary system commands of Linux

First, let's enlarge the permissions of the user directory inside the root of hdfs so that any user can read, write and execute inside user directory:

In [6]:
# hdfs dfs -chmod 777 /user

Note that, 777 is the most relaxed permission set and in production environments, it may be harmful: Think before you do it

Now let's add a directory to hdfs as /user/data

In [7]:
# hdfs dfs -mkdir /user/data

You can check that the directory is created checking the ambari UI

And now, let's "put" our data inside docker container into the hdfs, let's do it for the ncdc data:

In [ ]:
# hdfs dfs -mkdir /user/data/ncdc

In [3]:
# hdfs dfs -put /data/ncdc/txt /user/data/ncdc/

You can view the files are uploaded through Ambari UI or a command from docker shell: 

In [ ]:
# hdfs dfs -ls /data/ncdc/txt

Some examples to other commands that can be used with "hdfs dfs" are rm, cp, du and get

You can have more information on hdfs commands at:
https://hortonworks.com/hadoop-tutorial/using-commandline-manage-files-hdfs/

and using HDFS from Ambari at:

https://hortonworks.com/tutorial/hadoop-tutorial-getting-started-with-hdp/section/2/

Let's import other datasets:

In [ ]:
# hdfs dfs -mkdir /user/data/imdb

In [ ]:
# hdfs dfs -put /data/imdb/ /user/data/imdb/

In [ ]:
# hdfs dfs -mkdir /user/data/ngrams

In [ ]:
# hdfs dfs -put /data/ngrams/txt /user/data/ngrams/

And the code ...

In [ ]:
# hdfs dfs -put /data/codes /user/data/

Note that, all configuration parameters - like the replication factor or the hostnames of the slaves, or the hostname of the master node -  are set inside a few simple xml files such as:

- core-site.xml
- hadoop-env.sh
- hdfs-site.xml
- mapred-site.xml
- slaves

under $HADOOP_HOME/etc/hadoop

$HADOOP_HOME path may differ among systems

### Map Reduce Job

We will go through two examples using the first 10 years of the ncdc weather data set:

- Word Count
- Max Temperature

We should keep open a shell windows inside the sandbox-hdp

Either

`ssh -p 2223 root@localhost:8888`

or

on the browser navigate to:

`localhost:4200`

The root password is set as hadoopbda505 (if it doesn't work, try "hadoop")

After login, change hdfs user by:

`su hdfs`

#### Word Count on NCDC Dataset

This example is the "hello world" of map reduce and also cited inside the official documentation

We will get the total line count of all 10 files in the dataset

We will first implement the map reduce job as a unix command with pipes

If this works right, we will run it using hadoop streaming

Two versions will be run:
- One with a mapper and reducer,
- And the other with a mapper, combiner and reducer

In larger datasets run on many nodes, it is good practice to use a combiner so that the network traffic between the mapper and reducer is minimized

Note that all codes/commands must accept data from stdin and emit the result to stdout
It is best to delimit fields by tab

The mapper phase creates a key value pair out of the data. In the wc example, the original files are send to stdout 

"head" command exists just in order to limit the visible output and will not be a part of the mapper

In [1]:
# cat /data/ncdc/txt/* | head

Next, the reducer will count the lines

In [ ]:
# cat /data/ncdc/txt/* | wc -l
# 57920

Now we will convert this into a simple map reduce job:

- Note that the output folder should not exist, so flush it before running the job
- If the command takes parameters, wrap it around single or double quotes
- If the command includes pipes, do it like:
- `bash -c "your command | second command"`

- Note that input and output paths are inside the hdfs
- Mapper and reducer paths are inside the main filesystem
- Run these commands inside the docker shell:

`hdfs dfs -rm -r -f /user/data/ncdc/output*`

```bash
hadoop jar /usr/hdp/2.6.3.0-235/hadoop-mapreduce/hadoop-streaming.jar \
-input /user/data/ncdc/txt \
-output /user/data/ncdc/output \
-mapper /bin/cat \
-reducer '/usr/bin/wc -l'

```

If we have this line, everything went ok!:

`17/12/23 04:11:33 INFO streaming.StreamJob: Output directory: /user/data/ncdc/output`

Now we can check the output:

`hdfs dfs -cat /user/data/ncdc/output/*`

57920

Now we will rewrite the job adding a combiner:

The combiner will take over the job of reducer: For each task, the word count is calculated
Now the reducer will just add the word counts!

From the shell, the "bc" command will do it for us:

First let's see what happens until the reducer:

In [6]:
# for file in /data/ncdc/txt/*; \ # each file is send to cat separately
                                  # and we see what mapper, combiner and reducer does
# do cat $file | \ # mapper
# wc -l; done # combiner

# Copy the whole command into sandbox (docker) shell:
# for file in /data/ncdc/txt/*; do cat $file; wc -l; done

`[hdfs@sandbox-hdp comtrade]$ for file in /data/ncdc/txt/*; do cat $file | wc -l; done # combiner
6565
6565
6554
6582
6561
5474
5463
6585
7571
0
`

Now, add the reducer:
- "paste -sd+" puts a "+" sign between the numbers
- bc will calculate this formula, and add the numbers

In [7]:
# for file in /data/ncdc/txt/*; do cat $file | wc -l; done | paste -sd+ | bc

# 57920

Now let's convert it to a mapreduce job:
First flush the output directory

`hdfs dfs -rm -r -f /user/data/ncdc/output*`

And then:

```bash

hadoop jar /usr/hdp/2.6.3.0-235/hadoop-mapreduce/hadoop-streaming.jar \
-input /user/data/ncdc/txt \
-output /user/data/ncdc/output2 \
-mapper /bin/cat \
-combiner '/usr/bin/wc -l' \
-reducer "bash -c 'paste -sd+ | bc'"

```

17/12/23 04:29:24 INFO streaming.StreamJob: Output directory: /user/data/ncdc/output2

And see the output:

`hdfs dfs -cat /user/data/ncdc/output2/*`

57920

#### Title Count Across Years Using IMDB Dataset

Before going over a more detailed problem, you are given an assignment:

You will write a map reduce job using the imdb dataset:
- The files at the sandbox-hdp filesystem reside at directory /data/imdb/split
- These files are the title.basics2.tsv file split into 5 files of 1e6 lines
- The same files are imported into the hdfs at directory /user/data/imdb/split

You should be user hdfs inside sandbox-hdp:
`su hdfs`

You can view the files inside hdfs using the command:
`hdfs dfs -ls /user/data/imdb/split`

`[hdfs@sandbox-hdp split]$ hdfs dfs -ls /user/data/imdb/split
Found 5 items
-rw-r--r--   1 hdfs hdfs   82470261 2017-12-23 03:54 /user/data/imdb/split/xaa
-rw-r--r--   1 hdfs hdfs   84850192 2017-12-23 03:54 /user/data/imdb/split/xab
-rw-r--r--   1 hdfs hdfs   85630727 2017-12-23 03:54 /user/data/imdb/split/xac
-rw-r--r--   1 hdfs hdfs   84405203 2017-12-23 03:54 /user/data/imdb/split/xad
-rw-r--r--   1 hdfs hdfs   45003089 2017-12-23 03:54 /user/data/imdb/split/xae
`

You job is to 
- Get the start year column of the files as the mapper
- Get the count of each year as the combiner
- Aggregate the count of years in each task as the reducer

You can use the "cut" command to get the necessary column. Note that default field delimiter for cut is "\t", the same as the files

You can view the initial rows of the file with head, so that you decide on which column to extract

For combiner and reducer, you can use a small but very talented tool called "q" inside sandbox. "q" uses sqlite as its backend, however, it does not need a database: It can work on columnar data fed from stdin. Usual sql statements just work!

Only you should write "-" for the "from" clause and fields are named as c1, c2, etc. You can use select, from, group by and aggeragete functions such as count() or sum().

Note that when feeding into combiner or reducer you should wrap the line inside quotes. But 
the statement itself needs quotes. So one of the quote pair should be single and the other should be double as such: -combiner 'q "select ........"'

The output path you provide must be non-existent. So either provide a new one or flush the existing one.

First try the mapper, combiner and reducer on the command line inside the docker shell:

`for file in /data/imdb/split/*; do your_mapper_command | head -3; echo; done`

1894
1892
1892

2000
2000
2000

2013
2011
2011

2014
1909
2014

2016
2016
2016



Then we add the combiner ("head" and "column" are there just for visual purposes):

`for file in /data/imdb/split/*; do your_mapper_command $file | \
q "your_combiner_sql_statement" | head -20 | column -c 150; done`

`Warning: column count is one - did you provide the correct delimiter?
1888 2		1891 7		1894 66		1897 798	1900 811	1903 1642	1906 508
1889 1		1892 9		1895 65		1898 1049	1901 924	1904 492	1907 553
1890 3		1893 2		1896 466	1899 893	1902 879	1905 307
Warning: column count is one - did you provide the correct delimiter?
1888 1		1895 3		1898 86		1901 155	1904 551	1907 823	1910 2633
1890 1		1896 105	1899 174	1902 224	1905 482	1908 1689	1911 3076
1894 4		1897 117	1900 180	1903 224	1906 524	1909 2016
Warning: column count is one - did you provide the correct delimiter?
1874 1		1888 2		1891 2		1896 250	1899 719	1902 702	1905 892
1878 1		1889 1		1894 25		1897 407	1900 823	1903 795	1906 822
1887 1		1890 1		1895 47		1898 603	1901 673	1904 783
Warning: column count is one - did you provide the correct delimiter?
1883 1		1896 4		1899 5		1904 1		1908 56		1911 426	1914 666
1890 1		1897 4		1900 9		1905 23		1909 1581	1912 612	1915 517
1891 1		1898 2		1903 1		1907 7		1910 1465	1913 473
Warning: column count is one - did you provide the correct delimiter?
1887 1	1897 16	1899 9	1901 7	1903 14	1905 2	1907 4	1909 8	1911 15	1913 78
1896 7	1898 30	1900 11	1902 3	1904 3	1906 3	1908 3	1910 10	1912 65	1914 49
`


And last, we add our reducer:

`for file in /data/imdb/split/*; do your_mapper_command $file | \
q "your_combiner_sql_statement"; done | \
q "your_reducer_sql_statement" | column -c 100`

`Warning: column count is one - did you provide the correct delimiter?
Warning: column count is one - did you provide the correct delimiter?
Warning: column count is one - did you provide the correct delimiter?
Warning: column count is one - did you provide the correct delimiter?
Warning: column count is one - did you provide the correct delimiter?
1874 1		1909 5420	1934 2701	1959 13273	1984 23243	2009 160879
1878 1		1910 6410	1935 2645	1960 14479	1985 24729	2010 178864
1883 1		1911 6439	1936 3081	1961 14573	1986 25012	2011 207730
1887 2		1912 8459	1937 3341	1962 13627	1987 26746	2012 229315
1888 5		1913 9562	1938 3191	1963 15111	1988 25883	2013 242918
1889 2		1914 9003	1939 2828	1964 16602	1989 27896	2014 248265
1890 6		1915 8428	1940 2396	1965 18361	1990 29530	2015 251666
1891 10		1916 6959	1941 2379	1966 19025	1991 30996	2016 253051
1892 9		1917 5524	1942 2277	1967 19709	1992 32802	2017 162619
1893 2		1918 4608	1943 2064	1968 18150	1993 35493	2018 11721
1894 95		1919 3974	1944 1895	1969 19139	1994 39719	2019 975
1895 115	1920 4425	1945 1855	1970 19248	1995 46688	2020 212
1896 832	1921 4177	1946 2248	1971 19507	1996 47151	2021 34
1897 1342	1922 3566	1947 2716	1972 18904	1997 51891	2022 25
1898 1770	1923 3004	1948 3296	1973 19964	1998 58519	2023 5
1899 1800	1924 3057	1949 4310	1974 19458	1999 62518	2024 3
1900 1834	1925 3288	1950 5374	1975 20122	2000 66473	2025 1
1901 1759	1926 3016	1951 6333	1976 19101	2001 74113	2026 1
1902 1808	1927 3100	1952 7119	1977 19296	2002 77652	2115 1
1903 2676	1928 3065	1953 7824	1978 19401	2003 86811	N 321491
1904 1830	1929 3149	1954 8276	1979 20030	2004 102515
1905 1706	1930 2778	1955 9502	1980 20921	2005 115374
1906 1857	1931 2779	1956 10716	1981 19853	2006 129299
1907 2481	1932 2783	1957 12085	1982 20390	2007 143551
1908 4274	1933 2635	1958 12564	1983 21264	2008 151624
`


Now we are ready to run it as a mapreduce job on hdfs and yarn, before that we flush the output  directory (or pass a non-existent one)

`hdfs dfs -rm -r /user/data/imdb/output*`

`rm: `/user/data/imdb/output*': No such file or directory`

And run you mapreduce job:

`hadoop jar /usr/hdp/2.6.3.0-235/hadoop-mapreduce/hadoop-streaming.jar \
-input /user/data/imdb/split \
-output /user/data/imdb/output \
-mapper "your_mapper_command" \
-combiner 'q "your_combiner_sql_statement"' \
-reducer 'q "your_reducer_sql_statement"'`

You can track the job from the link using your browser:

http://sandbox-hdp.hortonworks.com:8088

This is important in order to detect bugs and problems by viewing the logs

You are also given the exact link for the ongoing job:

`17/12/23 06:19:40 INFO mapreduce.Job: The url to track the job: http://sandbox-hdp.hortonworks.com:8088/proxy/application_1513995729254_0004/
17/12/23 06:19:40 INFO mapreduce.Job: Running job: job_1513995729254_0004
17/12/23 06:19:49 INFO mapreduce.Job: Job job_1513995729254_0004 running in uber mode : false
17/12/23 06:19:49 INFO mapreduce.Job:  map 0% reduce 0%
17/12/23 06:20:18 INFO mapreduce.Job:  map 5% reduce 0%
17/12/23 06:20:21 INFO mapreduce.Job:  map 12% reduce 0%
17/12/23 06:20:24 INFO mapreduce.Job:  map 20% reduce 0%
17/12/23 06:20:25 INFO mapreduce.Job:  map 22% reduce 0%
17/12/23 06:20:28 INFO mapreduce.Job:  map 32% reduce 0%
17/12/23 06:20:31 INFO mapreduce.Job:  map 42% reduce 0%
17/12/23 06:20:34 INFO mapreduce.Job:  map 48% reduce 0%
17/12/23 06:20:37 INFO mapreduce.Job:  map 54% reduce 0%
17/12/23 06:20:40 INFO mapreduce.Job:  map 62% reduce 0%
17/12/23 06:20:43 INFO mapreduce.Job:  map 67% reduce 0%
17/12/23 06:21:09 INFO mapreduce.Job:  map 73% reduce 0%`

We are done!:

17/12/23 06:21:47 INFO streaming.StreamJob: Output directory: /user/data/imdb/output

Let's first check whether output files exist:

`[hdfs@sandbox-hdp split]$ hdfs dfs -ls /user/data/imdb/output
Found 2 items
-rw-r--r--   1 hdfs hdfs          0 2017-12-23 06:21 /user/data/imdb/output/_SUCCESS
-rw-r--r--   1 hdfs hdfs       1617 2017-12-23 06:21 /user/data/imdb/output/part-00000`


And see our output with some formatting into columns:

`hdfs dfs -cat /user/data/imdb/output/* | column -c 30`

`1874	 1		1933	 2635		1982	 20390
1878	 1		1934	 2701		1983	 21264
1883	 1		1935	 2645		1984	 23243
1887	 2		1936	 3081		1985	 24729
1888	 5		1937	 3341		1986	 25012
1889	 2		1938	 3191		1987	 26746
1890	 6		1939	 2828		1988	 25883
1891	 10		1940	 2396		1989	 27896
1892	 9		1941	 2379		1990	 29530
1893	 2		1942	 2277		1991	 30996
1894	 95		1943	 2064		1992	 32802
1895	 115		1944	 1895		1993	 35493
1896	 832		1945	 1855		1994	 39719
1897	 1342		1946	 2248		1995	 46688
1898	 1770		1947	 2716		1996	 47151
1899	 1800		1948	 3296		1997	 51891
1900	 1834		1949	 4310		1998	 58519
1901	 1759		1950	 5374		1999	 62518
1902	 1808		1951	 6333		2000	 66473
1903	 2676		1952	 7119		2001	 74113
1904	 1830		1953	 7824		2002	 77652
1905	 1706		1954	 8276		2003	 86811
1906	 1857		1955	 9502		2004	 102515
1907	 2481		1956	 10716		2005	 115374
1908	 4274		1957	 12085		2006	 129299
1909	 5420		1958	 12564		2007	 143551
1910	 6410		1959	 13273		2008	 151624
1911	 6439		1960	 14479		2009	 160879
1912	 8459		1961	 14573		2010	 178864
1913	 9562		1962	 13627		2011	 207730
1914	 9003		1963	 15111		2012	 229315
1915	 8428		1964	 16602		2013	 242918
1916	 6959		1965	 18361		2014	 248265
1917	 5524		1966	 19025		2015	 251666
1918	 4608		1967	 19709		2016	 253051
1919	 3974		1968	 18150		2017	 162619
1920	 4425		1969	 19139		2018	 11721
1921	 4177		1970	 19248		2019	 975
1922	 3566		1971	 19507		2020	 212
1923	 3004		1972	 18904		2021	 34
1924	 3057		1973	 19964		2022	 25
1925	 3288		1974	 19458		2023	 5
1926	 3016		1975	 20122		2024	 3
1927	 3100		1976	 19101		2025	 1
1928	 3065		1977	 19296		2026	 1
1929	 3149		1978	 19401		2115	 1
1930	 2778		1979	 20030		N	 321491
1931	 2779		1980	 20921
1932	 2783		1981	 19853`

The same as the output from the shell command!

## Creating Scripts for MapReduce Job

### Maximum Temperature Example on NCDC Dataset

Now, using the same NCDC data set we'll recreate the maximum temperature example from the Elephant Book as a "Hadoop streaming" job:

Your computers have a century of the temperature data of USA. We will use just the first 10 years of this data

And for each year we will get the max temperature

We will first start with standard unix tools

I played a bit with the original script given as a part of the supplementary material for the Elephant Book

Note that there may be empty files and the job must have a remedy for this issue

The script is as follows:

```bash
#!/usr/bin/env bash

# adjusted by Serhat Cevikel to measure the time

#START=$(date +%s.%N)
path=$1
starty=$2
endy=$3

years=$(seq $starty $endy)


for year in $years
do
    filee="${path}/${year}"
  echo -ne `basename $year .gz`"\t"
  gunzip -c $filee | \ 
    awk '{ temp = substr($0, 88, 5) + 0;
           q = substr($0, 93, 1);
           if (temp !=9999 && q ~ /[01459]/ && temp > max) max = temp }
         END { print max }'
done
```

Missing temp values are coded as 9999 and they are excluded
"q" is a quality code and should be one of 0,1,4,5,9 to be included

The path to script inside sandbox-hdp is /data/codes/hadoop_max_temperature.sh

In my implementation it takes 3 parameters: the path to gz files, start year and end year

The data resides at /data/ncdc/gz/ and /data/ncdc/txt/

Let's do it for 1901 to 1925

```bash
/data/codes/hadoop_max_temperature.sh \
/data/ncdc/gz \
1901 1910
```

```
1901	317
1902	244
1903	289
1904	256
1905	283
1906	294
1907	283
1908	289
1909	278
1910	
```

Now let's transform it to a map reduce job 

In the map phase, the key value pairs are extracted from the data: the year and temp reading

In the reduce phase the max temp for each year is calculated

```R
#!/usr/bin/Rscript

con <- file("stdin")
#con <- file("1910")
liness <- readLines(con)
close(con)

year <- as.numeric(substr(liness, 16, 19))
temp <- as.numeric(substr(liness, 88, 92))
qq <- as.numeric(substr(liness, 93, 93))

output <- cbind(year, temp)

output <- output[temp != 9999 & qq %in% c(0, 1, 4, 5, 9),]

for (i in seq_along(output[,1]))
{
    pasted <- paste(output[i,], collapse = "\t")
    cat(sprintf("%s\n", pasted))
}
```                    

The code accepts input from stdin so can be used similar to the previous one - before Hadoop

The path to code inside sandbox-hdp is /data/codes/mapper.R

`cat /data/ncdc/txt/{1901..1910} | /data/codes/mapper.R | head`
```
1901	-78
1901	-72
1901	-94
1901	-61
1901	-56
1901	-28
1901	-67
1901	-33
1901	-28
1901	-33
```

The reducer code is as follows:

```R
#!/usr/bin/Rscript

con <- file("stdin")
#com <- file("mapped")
liness <- readLines(con)
close(con)

keyval <- list()

for (i in seq_along(liness))
{
    linex <- unlist(strsplit(liness[i], split = "\t"))
    key <- linex[1]
    val <- as.numeric(linex[2])

    cur.maxval <- keyval[[key]]

    if (is.null(cur.maxval))
    {   
        keyval[[key]] <- val 
    }
    else
    {   
        if (val > cur.maxval)
        {
            keyval[[key]] <- val 
        }
    }
}

keys <- as.numeric(names(keyval))
vals <- as.numeric(unlist(keyval))

output <- matrix(c(keys, vals), ncol = 2)
output <- output[order(keys),, drop = F]

for (i in seq_along(output[,1]))
{
    pasted <- paste(output[i,], collapse = "\t")
    cat(sprintf("%s\n", pasted))
}
```

The path to the reducer code is /home/bda505/mef-bigdata/week_07/codes/reducer.R

`cat /data/ncdc/txt/{1901..1910} | /data/codes/mapper.R | /data/codes/reducer.R`

```
1901	317
1902	244
1903	289
1904	256
1905	283
1906	294
1907	283
1908	289
1909	278
```

Now we can run the map reduce job. Note that we have to pass the custom script files via "-file" option so that all nodes can run it:

```bash
hadoop jar /usr/hdp/2.6.3.0-235/hadoop-mapreduce/hadoop-streaming.jar \
-input /user/data/ncdc/txt \
-output /user/data/ncdc/output16 \
-mapper /data/codes/mapper.R \
-reducer /data/codes/reducer.R \
-file /data/codes/mapper.R \
-file /data/codes/reducer.R
```

Or we can run it with a combiner (the same script as the reducer for this case):

```bash

hadoop jar /usr/hdp/2.6.3.0-235/hadoop-mapreduce/hadoop-streaming.jar \
-input /user/data/ncdc/txt \
-output /user/data/ncdc/output16 \
-mapper /data/codes/mapper.R \
-combiner reducer.R \
-reducer /data/codes/reducer.R \
-file /data/codes/mapper.R \
-file /data/codes/reducer.R

```